# Experiment 3: Risk of Regret

> Want to cut losses? The workload changed! A new technology has just appeared! The original topology simply doesn’t work as predicted!
Does it make sense to speed up migration to new platform?
Will my datacenter be at risk once deployed?

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import radice

FIGURE_PATH = 'figures/03-regret'

os.makedirs(FIGURE_PATH, exist_ok=True)
radice.set()

## Different Workloads
Data is obtained by running the following command:
```bash
bin/radice run -r 4096 -p 80 portfolios/workload.yml -P portfolio=workload
```

In [ ]:
risk = pd.read_parquet('data/risk/portfolio=workload', columns=['workload', 'seed', 'timestamp', 'id', 'cost'], read_dictionary=['id'])

In [ ]:
risk_pm = radice.compute_monthly_risk(risk, keys=['workload'], observed=True)
risk_pm.groupby(["workload"])['cost'].describe()

In [ ]:
fig, ax = plt.subplots(figsize=radice.figsize(1.35))

order = [
    'baseline',
    'materna',
    'bitbrains',
    'azure',
]

ax = sns.boxplot(data=risk_pm, x='cost', y='workload', ax=ax, order=order, showfliers=False, showmeans=True, color='C0')

ax.set_ylabel("Workload")
ax.set_xlabel("Risk per Month (€)")
ax.set_yticklabels(['Baseline', 'Materna', 'Bitbrains', 'Azure'])
ax.set_xlim(left=0)
ax.xaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))

fig.savefig(os.path.join(FIGURE_PATH, 'trace-comparison.pdf'))

In [ ]:
data = pd.concat([
    radice.compute_monthly_risk(risk[risk['workload'] == 'baseline'], keys=['id']).assign(workload='baseline'),
    radice.compute_monthly_risk(risk[risk['workload'] == 'materna'], keys=['id']).assign(workload='materna'),
    radice.compute_monthly_risk(risk[risk['workload'] == 'bitbrains'], keys=['id']).assign(workload='bitbrains'),
    radice.compute_monthly_risk(risk[risk['workload'] == 'azure'], keys=['id']).assign(workload='azure'),
])
data.groupby(["workload", "id"])['cost'].describe()

In [ ]:
fig, ax = radice.plot_risk_factors_horizontal(data, hue='workload', hue_order=['baseline', 'materna', 'bitbrains', 'azure'], bar=True)

legend = ax[0].get_legend()
legend.set_title("Workload")
labels = ['Baseline', 'Materna', 'Bitbrains', 'Azure']

for i, txt in enumerate(legend.get_texts()):
    txt.set_text(labels[i])

ax[1].legend([],[], frameon=False)

fig.savefig(os.path.join(FIGURE_PATH, 'risk-profile-traces.pdf'))

In [ ]:
risk_pm_azure = radice.compute_monthly_risk(risk[risk['workload'] == 'azure'])
risk_pm_azure.groupby(["id"])['cost'].describe()

In [ ]:
fig, ax = radice.plot_risk_factors_horizontal(risk_pm_azure, bar=True)
fig.savefig(os.path.join(FIGURE_PATH, 'risk-profile-azure.pdf'))

In [ ]:
risk_pm_materna = radice.compute_monthly_risk(risk[risk['workload'] == 'materna'])
risk_pm_materna.groupby(["id"])['cost'].describe()

In [ ]:
fig, _ = radice.plot_risk_factors_horizontal(risk_pm_materna)
fig.savefig(os.path.join(FIGURE_PATH, 'risk-profile-materna.pdf'))

In [ ]:
risk_pm_bitbrains = radice.compute_monthly_risk(risk[risk['workload'] == 'bitbrains'])
risk_pm_bitbrains.groupby(["id"])['cost'].describe()

In [ ]:
fig, _ = radice.plot_risk_factors_horizontal(risk_pm_bitbrains)
fig.savefig(os.path.join(FIGURE_PATH, 'risk-profile-bitbrains.pdf'))
